In [28]:
import os
import numpy as np
import pandas as pd
import alpaca_trade_api as tradeapi
from MCForecastTools import MCSimulation
from pathlib import Path
from dotenv import load_dotenv
import yfinance as yf
import hvplot.pandas
import panel as pn
from panel.interact import interact
import plotly.express as px
pn.extension("plotly")
load_dotenv()
%matplotlib inline

In [3]:
df_sp500 = pd.read_csv("DATA SOURCES/sp500_historical.csv", index_col="Date", infer_datetime_format=True, parse_dates=True)
df_dowjones = pd.read_csv("DATA SOURCES/dowjones_historical.csv", index_col="Date", infer_datetime_format=True, parse_dates=True)
df_nasdaq = pd.read_csv("DATA SOURCES/nasdaq_historical.csv", index_col="Date", infer_datetime_format=True, parse_dates=True)
df_gold = pd.read_csv("DATA SOURCES/gold_historical.csv", index_col="Date", infer_datetime_format=True, parse_dates=True)
df_silver = pd.read_csv("DATA SOURCES/silver_historical.csv", index_col="Date", infer_datetime_format=True, parse_dates=True)
df_crudeoil = pd.read_csv("DATA SOURCES/crudeoil_historical.csv", index_col="Date", infer_datetime_format=True, parse_dates=True)

In [4]:
df_sp500.drop(columns=["Open","High","Low","Vol.","Change %"],inplace=True)
df_dowjones.drop(columns=["Open","High","Low","Vol.","Change %"],inplace=True)
df_nasdaq.drop(columns=["Open","High","Low","Vol.","Change %"],inplace=True)
df_gold.drop(columns=["Open","High","Low","Vol.","Change %"],inplace=True)
df_silver.drop(columns=["Open","High","Low","Vol.","Change %"],inplace=True)
df_crudeoil.drop(columns=["Open","High","Low","Vol.","Change %"],inplace=True)

In [5]:
df_sp500.rename(columns={"Price":"sp500"},inplace=True)
df_dowjones.rename(columns={"Price":"dowjones"},inplace=True)
df_nasdaq.rename(columns={"Price":"nasdaq"},inplace=True)
df_gold.rename(columns={"Price":"gold"},inplace=True)
df_silver.rename(columns={"Price":"silver"},inplace=True)
df_crudeoil.rename(columns={"Price":"crudeoil"},inplace=True)

In [6]:
df_commodities = pd.concat([df_sp500,df_dowjones,df_nasdaq,df_gold], join="inner", axis="columns")
df_commodities.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 432 entries, 2021-01-21 to 1985-02-01
Data columns (total 4 columns):
sp500       432 non-null object
dowjones    432 non-null object
nasdaq      432 non-null object
gold        432 non-null object
dtypes: object(4)
memory usage: 16.9+ KB


In [7]:
df_commodities = df_commodities.apply(lambda x: x.str.replace(",",""))
df_commodities = df_commodities.astype("float",inplace=True)
df_commodities.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 432 entries, 2021-01-21 to 1985-02-01
Data columns (total 4 columns):
sp500       432 non-null float64
dowjones    432 non-null float64
nasdaq      432 non-null float64
gold        432 non-null float64
dtypes: float64(4)
memory usage: 16.9 KB


In [8]:
df_commodities = pd.concat([df_commodities,df_silver,df_crudeoil],join="inner",axis="columns")
df_commodities.head()

,sp500,dowjones,nasdaq,gold,silver,crudeoil
Date,,,,,,
2021-01-21,3714.24,29982.62,13070.70,1850.3,26.914,52.20
2020-12-20,3756.07,30606.48,12888.28,1895.1,26.412,48.52
2020-11-20,3621.63,29638.64,12198.74,1780.9,22.593,45.34
2020-10-20,3269.96,26501.60,10911.59,1887.2,23.799,35.79
2020-09-20,3363.00,27781.70,11167.51,1902.6,23.627,40.22


In [9]:
df_commodities_return = df_commodities.pct_change()
df_commodities_corr = df_commodities_return.corr()
df_commodities_corr

,sp500,dowjones,nasdaq,gold,silver,crudeoil
sp500,1.000000,0.958601,0.861208,-0.045192,0.165771,0.185831
dowjones,0.958601,1.000000,0.772535,-0.064367,0.144214,0.162589
nasdaq,0.861208,0.772535,1.000000,-0.032271,0.145273,0.150861
gold,-0.045192,-0.064367,-0.032271,1.000000,0.711512,0.162036
silver,0.165771,0.144214,0.145273,0.711512,1.000000,0.250716
crudeoil,0.185831,0.162589,0.150861,0.162036,0.250716,1.000000


In [27]:
prices_commodities = df_commodities[["sp500","gold","silver","crudeoil"]].hvplot.line(xlabel="Date", ylabel="Monthly Close Price")
prices_gold = df_commodities[["sp500","gold"]].hvplot.line(xlabel="Date", ylabel="Monthly Close Price")
prices_silver = df_commodities[["sp500","silver"]].hvplot.line(xlabel="Date", ylabel="Monthly Close Price")
prices_crudeoil = df_commodities[["sp500","crudeoil"]].hvplot.line(xlabel="Date", ylabel="Monthly Close Price")

In [24]:
df_comm_rolling_corr = df_commodities_return.rolling(window=12).corr().dropna()
df_comm_rolling_corr.reset_index(inplace=True)
df_comm_rolling_corr.set_index("Date",inplace=True)
df_comm_rolling_corr.head()


,level_1,sp500,dowjones,nasdaq,gold,silver,crudeoil
Date,,,,,,,
2020-01-20,sp500,1.000000,0.986715,0.984297,0.304379,0.650256,0.713023
2020-01-20,dowjones,0.986715,1.000000,0.961692,0.200399,0.566715,0.729946
2020-01-20,nasdaq,0.984297,0.961692,1.000000,0.364903,0.688587,0.677368
2020-01-20,gold,0.304379,0.200399,0.364903,1.000000,0.699218,0.051296
2020-01-20,silver,0.650256,0.566715,0.688587,0.699218,1.000000,0.557160


In [130]:
gold_corr = df_comm_rolling_corr[df_comm_rolling_corr['level_1'] == 'gold'].sp500
crude_oil_corr = df_comm_rolling_corr[df_comm_rolling_corr['level_1'] == 'crudeoil'].sp500
silver_corr = df_comm_rolling_corr[df_comm_rolling_corr['level_1'] == 'silver'].sp500
dowjones_corr = df_comm_rolling_corr[df_comm_rolling_corr['level_1'] == 'dowjones'].sp500

In [95]:
tickers = [
          "BTC-USD",
          "ETH-USD",
          "^GSPC"
          ]

df_crypto = yf.download(tickers,
start="1980-01-01",
end="2021-01-29",
progress=False,
interval = "1mo",
groupby = "ticker"
                )

In [96]:
df_crypto.drop(columns=['Open', 'High', 'Low', 'Volume', 'Adj Close'], inplace=True)

In [97]:
df_crypto.rename(columns=
          {"BTC-USD":"btc",
             "ETH-USD":"eth",
             "^GSPC":"sp500",
            },inplace=True)

In [98]:
df_crypto.columns = df_crypto.columns.droplevel(level=0)

In [99]:
df_crypto.dropna(inplace=True)

In [100]:
df_crypto_return = df_crypto.pct_change()

In [101]:
df_crypto_corr = df_crypto_return.corr()

In [102]:
df_crypto_rolling_corr = df_crypto_return.rolling(window=12).corr().dropna()
df_crypto_rolling_corr.reset_index(inplace=True)
df_crypto_rolling_corr.set_index("Date",inplace=True)
df_crypto_rolling_corr.head()

,level_1,btc,eth,sp500
Date,,,,
2016-09-01,btc,1.000000,-0.177658,0.287968
2016-09-01,eth,-0.177658,1.000000,-0.207934
2016-09-01,sp500,0.287968,-0.207934,1.000000
2016-10-01,btc,1.000000,-0.199912,-0.077153
2016-10-01,eth,-0.199912,1.000000,-0.141507


In [103]:
btc_corr = df_crypto_rolling_corr[df_crypto_rolling_corr['level_1'] == 'btc'].sp500
eth_corr = df_crypto_rolling_corr[df_crypto_rolling_corr['level_1'] == 'eth'].sp500


In [113]:
def comm_correlation():
    comm_corr = df_commodities_corr.hvplot.heatmap(title = "Commodities and Indices Correlation")    
    return comm_corr
def crypto_correlation():
    crypto_corr = df_crypto_corr.hvplot.heatmap(title = "Crytocurrencies and Indices Correlation")    
    return crypto_corr

list_of_commodities = ["gold","silver","crudeoil"]
def commodities_price(commodities):
    comm_price = df_commodities[["sp500",commodities]].hvplot.line(title = f"Price of {commodities} and S&P500",
                                                                           xlabel="Date", ylabel="Monthly Close Price",ylim=(0,4000))
    return comm_price
def commodities_correlation(commodities):
    comm_corr = df_comm_rolling_corr[df_comm_rolling_corr['level_1'] == commodities].sp500
    return comm_corr.hvplot.line(title = f"{commodities} & S&P500 12 month Rolling Correlation", xlabel="Date", ylabel="Correlation")


list_of_crypto = ["btc","eth"]
def cryptocurrency_price(crypto):
    crypto_price = df_crypto[["sp500",crypto]].hvplot.line(title = f"Price of {crypto} and S&P500",
                                                                           xlabel="Date", ylabel="Monthly Close Price")
    return crypto_price
def cryptocurrency_corr(crypto):
    crypto_corr = df_crypto_rolling_corr[df_crypto_rolling_corr['level_1'] == crypto]
    return crypto_corr.sp500.hvplot.line(title = f"{crypto} and S&P500 12 month Rolling Correlation",
                                                                           xlabel="Date", ylabel="Correlation")


In [128]:
comm_summary = pn.Column("#S&P 500 and Commodities Summary", prices_commodities, comm_correlation())
crypto_summary = pn.Column("#S&P 500 and Cryptocurrencies Summary", df_crypto.hvplot.line(xlabel="Date", ylabel="Monthly Close Price"), crypto_correlation())
comm_analysis = pn.Column("#Commodities Analysis", interact(commodities_price, commodities=list_of_commodities), interact(commodities_correlation,commodities=list_of_commodities))
crypto_analysis = pn.Column("#Cryptocurrencies Analysis", interact(cryptocurrency_price, crypto=list_of_crypto), interact(cryptocurrency_corr,crypto=list_of_crypto))
panel1=pn.Tabs(
    ("Commodities Summary",comm_summary), ("Commodities Analysis",comm_analysis))
panel2=pn.Tabs(("Cryptocurrencies Summary", crypto_summary),("Cryptocurrencies Analysis",crypto_analysis))

In [129]:
panel1.servable()

Tabs
    [0] Column
        [0] Markdown(str)
        [1] HoloViews(NdOverlay)
        [2] HoloViews(HeatMap)
    [1] Column
        [0] Markdown(str)
        [1] Column
            [0] Column
                [0] Select(name='commodities', options=['gold', 'silver', ...], value='gold')
            [1] Row
                [0] HoloViews(NdOverlay, name='interactive38290')
        [2] Column
            [0] Column
                [0] Select(name='commodities', options=['gold', 'silver', ...], value='gold')
            [1] Row
                [0] HoloViews(Curve, name='interactive38397')

From 1985 to early 2000, gold was used to hedge against market downturn. Hence, gold price moved in opposite direction of S&P500 index.
This is also shown as negative (inverse) correlation until early 2000. The correlation started to change to be more positive due to creation of paper gold and commodities market in 1970's. However, it took some time for the paper gold and commdities futures market to be popularized; therefore, the transition was not apparent until mid 2000's. Since 2007, we noticed that Crude Oil has become more collerated to S&P500, possibly due to the start of oil price war.



In [127]:
panel2.servable()

Tabs(active=1)
    [0] Column
        [0] Markdown(str)
        [1] HoloViews(NdOverlay)
        [2] HoloViews(HeatMap)
    [1] Column
        [0] Markdown(str)
        [1] Column
            [0] Column
                [0] Select(name='crypto', options=['btc', 'eth'], value='btc')
            [1] Row
                [0] HoloViews(NdOverlay, name='interactive34363')
        [2] Column
            [0] Column
                [0] Select(name='crypto', options=['btc', 'eth'], value='btc')
            [1] Row
                [0] HoloViews(Curve, name='interactive34470')

Cryptocurrencies has weak correlation to S&P500.